In [0]:
%fs ls /public/foot_mercato

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW players_informations
USING JSON
OPTIONS (path = '/public/foot_mercato/players_info_20250728.jsonl', multiline = false);


CREATE OR REPLACE TEMPORARY VIEW transferts_informations
USING JSON
OPTIONS (path = '/public/foot_mercato/transfert_info_20250728.jsonl', multiline = false);


CREATE OR REPLACE TEMPORARY VIEW clubs
USING CSV
OPTIONS (
  path "/public/foot_mercato/clubs_cleaned.csv",
  header "true",
  inferSchema "true"
)



In [0]:
%sql
SELECT * from players_informations
LIMIT 1

In [0]:
%sql
SELECT * from transferts_informations
LIMIT 1

In [0]:
%sql
CREATE TEMPORARY VIEW transfert_cleaned as
WITH transfert_player_cte AS (
  SELECT
      ti.transfer_id,
      year(to_date(date)) as year,
      pi.id, 
      pi.name,
      ti.age,
      pi.position.name as position,
      ac.name as provenance,
      dc.name as arrival,
      ti.season,
      CASE 
        WHEN ti.fee IN ('-', '','Free Transfer','draft') THEN 0
        WHEN ti.fee LIKE '%M' THEN CAST(REPLACE(REPLACE(ti.fee, '€', ''), 'M', '') AS INT) * 1000000
        WHEN ti.fee LIKE '%K' THEN CAST(REPLACE(REPLACE(ti.fee, '€', ''), 'K', '') AS INT) * 1000
        WHEN ti.fee LIKE '€%' THEN CAST(ti.fee AS INT)
        ELSE
          ti.fee
       END AS fee_cleaned,
      ti.fee
    FROM transferts_informations as ti
      JOIN players_informations as pi 
        ON pi.id = ti.player_id
      JOIN clubs as ac
        ON ti.source_club_id = ac.id
      JOIN clubs as dc
        ON ti.destination_club_id = dc.id
      
)
SELECT * from transfert_player_cte
WHERE transfert_player_cte.fee_cleaned != '?'

 






In [0]:
%sql
SELECT 
    year,
    FORMAT_NUMBER(sum(cast(fee_cleaned AS INT)),0) as total_fees_year,
    count(*) as total_transfert
FROM transfert_cleaned
GROUP BY year
ORDER BY year DESC

In [0]:
%sql
WITH transfert_by_club_cte AS(
SELECT 
    arrival,
    sum(cast(fee_cleaned AS INT)) as total_fees,
    count(*) as total_transfert
FROM transfert_cleaned
GROUP BY arrival
)
SELECT
  arrival,
  FORMAT_NUMBER(total_fees,0) as fees_club_total,
  total_transfert, 
  FORMAT_NUMBER(total_fees/total_transfert,0) as avg_tr
FROM transfert_by_club_cte
ORDER BY total_fees DESC 